In [1]:
import os
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG
from tensorflow import keras
from keras import applications
from keras import optimizers
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.utils.np_utils import to_categorical
from keras import layers, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau


train_path = "../images/train/"
test_path = "../images/test/"
img_width, img_height = 224, 224
channels = 3
batch_size = 64
num_images = 100
image_arr_size = img_width * img_height * channels


In [2]:

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    rotation_range= 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    validation_split=0.2,

)

valid_datagen = ImageDataGenerator(
    rescale= 1./255,
    validation_split=0.2,
)


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='training',
    shuffle= True,
    seed= 1337
)

valid_generator = valid_datagen.flow_from_directory(
    train_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='validation',
    shuffle= True,
    seed= 1337
)

Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.


In [3]:
num_classes = len(train_generator.class_indices)
train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = valid_generator.classes
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_generator.filenames)
nb_valid_samples = len(valid_generator.filenames)

In [4]:
test_datagen = ImageDataGenerator(rescale=1/255.0)
test_generator = test_datagen.flow_from_directory(
                                    test_path,
                                    target_size=(img_width, img_height),
                                    color_mode= 'rgb',
                                    batch_size=batch_size,
                                    class_mode= 'categorical',
                                    shuffle=True,)
test_generator.reset()

Found 8580 images belonging to 120 classes.


# Model

# 1. basic CNN

In [5]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [7]:
isp = (img_width, img_height, channels)


model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),input_shape=isp))
model.add(Flatten())
model.add(layers.Dense(10,kernel_initializer='he_uniform'))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120,activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=model.fit(train_generator, steps_per_epoch=50, epochs=3, verbose= 1, validation_data= valid_generator)

Epoch 1/3
50/50 [==============================] - 83s 2s/step - loss: 7.6398 - accuracy: 0.0069 - val_loss: 4.7875 - val_accuracy: 0.0083
Epoch 2/3
50/50 [==============================] - 83s 2s/step - loss: 4.7879 - accuracy: 0.0056 - val_loss: 4.7875 - val_accuracy: 0.0083
Epoch 3/3
50/50 [==============================] - 82s 2s/step - loss: 4.7877 - accuracy: 0.0084 - val_loss: 4.7875 - val_accuracy: 0.0083


In [8]:
pred = model.predict(test_generator, verbose=2)
pred = np.argmax(pred,axis=1)

135/135 - 45s


In [9]:
test_accuracy = np.sum(np.array(pred))/len(pred)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 93.0000%


In [10]:
score = model.evaluate(test_generator, verbose=2)

135/135 - 46s - loss: 4.7878 - accuracy: 0.0058


In [11]:
print('score={0}'.format(score))

score=[4.787779331207275, 0.005827505607157946]
